In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import gdown
import zipfile
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

In [ ]:
model = load_model('/content/drive/MyDrive/Orbit_final_project/bestmodel8')
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 model_1_input (InputLayer)  [(None, 224, 224, 3)]     0         
                                                                 
 model_1 (Functional)        (None, 1280)              2257984   
                                                                 
 dense (Dense)               (None, 1024)              1311744   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                             

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_dir = '/content/drive/MyDrive/Orbit_final_project/data_split/train'
test_dir = '/content/drive/MyDrive/Orbit_final_project/data_split/test'
val_dir = '/content/drive/MyDrive/Orbit_final_project/data_split/val'

test_datagen = ImageDataGenerator(
                  rescale=1./255
)

train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=45,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      brightness_range=[0.6, 1.5],
      horizontal_flip=True,
      fill_mode='nearest')

In [ ]:
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)


test_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    batch_size=434,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    batch_size=434,
    class_mode='categorical'
)

Found 2766 images belonging to 11 classes.
Found 434 images belonging to 11 classes.
Found 434 images belonging to 11 classes.


In [ ]:
model.evaluate(train_generator)

44/44 [==============================] - 202s 5s/step - loss: 0.4291 - accuracy: 0.9364


[0.42912811040878296, 0.9363701939582825]

In [ ]:
model.evaluate(test_generator)

1/1 [==============================] - 30s 30s/step - loss: 0.3688 - accuracy: 0.9631


[0.36877384781837463, 0.9631336331367493]

In [ ]:
model.evaluate(val_generator)

1/1 [==============================] - 28s 28s/step - loss: 0.3688 - accuracy: 0.9631


[0.368773877620697, 0.9631336331367493]

In [ ]:
import numpy as np

In [ ]:
label = list(train_generator.class_indices.keys())
label

['bika_ambon',
 'cucur',
 'dadar_gulung',
 'data_selain_kue',
 'klepon',
 'kue_cubit',
 'kue_lupis',
 'kue_pukis',
 'kue_sus',
 'onde_onde',
 'putu_ayu']

In [ ]:
def predict(path, model):
    img = tf.keras.utils.load_img(path, target_size=(224, 224, 3))
    img = np.array(img)
    img = img/255
    pred = model.predict(img[np.newaxis, ...])
    score = np.max(pred)
    classes_idx = np.argmax(pred)
    classes = label[classes_idx]
    return classes, score

In [ ]:
test = [i for i in os.listdir('/content') if "jpeg" in i]
test

['WhatsApp Image 2022-01-15 at 22.19.36.jpeg',
 'WhatsApp Image 2022-01-15 at 22.19.36 (2).jpeg',
 'WhatsApp Image 2022-01-15 at 22.19.36 (3).jpeg',
 'WhatsApp Image 2022-01-15 at 22.19.36 (1).jpeg',
 'WhatsApp Image 2022-01-15 at 22.19.36 (4).jpeg',
 'WhatsApp Image 2022-01-15 at 22.19.36 (5).jpeg']

In [ ]:
for tst in test:
    print(predict(tst, model))

('dadar_gulung', 0.83721393)
('dadar_gulung', 0.8754961)
('dadar_gulung', 0.6807771)
('dadar_gulung', 0.9724275)
('data_selain_kue', 0.8072602)
('dadar_gulung', 0.48071653)


In [ ]:
predict('/content/WhatsApp Image 2022-01-15 at 22.19.36 (2).jpeg', model)

('dadar_gulung',
 0.8754961,
 array([[6.0668826e-05, 8.9379258e-02, 8.7549609e-01, 6.7122830e-03,
         8.9629451e-05, 1.3821803e-04, 3.5281030e-03, 1.5642622e-02,
         2.5639567e-03, 4.4879180e-06, 6.3846707e-03]], dtype=float32))

In [ ]:
from tensorflow import saved_model
import pathlib

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model('/content/drive/MyDrive/Orbit_final_project/bestModel8/checkpoint_08')
tflite_model = converter.convert()
 
tflite_model_file = pathlib.Path('bestModel8.tflite')
tflite_model_file.write_bytes(tflite_model)

18353612